In [5]:
import gradio as gr
import time
import os
import sys
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
from aip import AipSpeech
import speech_recognition as sr
import pygame
 
APP_ID = '117045037'
API_KEY = 'QTHyrEY1qfGVyHBZNRgSQXFy'
SECRET_KEY = 'Xnh8clQsR8cZu9kDiVe4cOp5qrBmQnbb'
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)
        
def init():
    global stamp, wj_flag, item
    wj_flag = False
    stamp = time.localtime()
    item = []

init()
src = 'C:\\Users\\26073\\Workspace\\detector\\'

def synthesize_speech(text, audio_filename):
    # 调用百度TTS接口进行合成
    result = client.synthesis(text, 'zh', 1, {
        'vol': 5,  # 音量 0-15
        'per': 4,  # 选择发音人，0-普通女声，1-普通男声，3-情感合成女声，4-情感合成男声
        'spd': 5,  # 语速 0-15
        'pit': 5   # 音调 0-15
    })
    if not isinstance(result, dict):
        save_path = src + audio_filename
        with open(save_path, 'wb') as f:
            f.write(result)  # 保存音频到文件
        print(f"语音合成成功，音频保存为 {save_path}")
        play_audio(save_path)
    else:
        print("语音合成失败")

def play_audio(file_path):
    pygame.mixer.init()
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()
    init()

def calculate(img):
    global wj_flag
    img_name = "demo" + time.strftime("%Y%m%d%H%M%S", stamp)
    img.save(src + img_name + ".jpg", format="JPEG")
    prj_name = src + "_" + img_name
    model = YOLO(src + 'yolov8x-oiv7.pt')
    model.predict(
        source = src + img_name + ".jpg",
        save = True,
        save_txt = True,
        project = src,
        name = prj_name,
    )
    if os.path.exists(prj_name + "/labels/" + img_name + ".txt"):
        img = Image.open(prj_name + "/" + img_name + ".jpg")
        file_ori = open(prj_name + "/labels/" + img_name + ".txt", "r")
        lines_ori = file_ori.readlines()
        with open(src + "set-zh.txt", 'r', encoding='utf-8') as file_ref:
            lines_ref = file_ref.readlines()
        for line in lines_ori:
            line1 = line.split()
            num = int(line1[0])
            item.append(lines_ref[num].strip())
        file_ori.close()
        file_ref.close()
        
        uitem = list(set(item))
        
        wj = ["手枪", "炸弹", "凿子","匕首", "钻头", "叉子", "磨碎机", "厨房刀具", "刀", "导弹", "钉子", "步枪", "武器"]
        for it in range(len(uitem)):
            if uitem[it] in wj:
                uitem[it] += "【违禁品!】"
                wj_flag = True
        text = "共计" + str(len(uitem)) + "个物品"
        if wj_flag:
            text += "，其中包含违禁品！"
        else:
            text += "，未发现违禁品！"
        audio_filename="demo" + time.strftime("%Y%m%d%H%M%S", stamp) + ".mp3"
        synthesize_speech(text, audio_filename)
        
        return [img, "   ".join(uitem)]
    else:
        text = "未检测到物品！"
        audio_filename="demo" + time.strftime("%Y%m%d%H%M%S", stamp) + ".mp3"
        synthesize_speech(text, audio_filename)
        
        return [img, ""]

iface = gr.Interface(
    fn=calculate,
    inputs=[
        gr.Image(type="pil",label="原图"),
    ],
    outputs=[
        gr.Image(type="pil", label="结果"),
        gr.Textbox(label="物品"),
    ],
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://5ce79b5ab574dd9d03.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



image 1/1 C:\Users\26073\Workspace\detector\demo20250118212217.jpg: 640x640 1 Coffee cup, 1 Desk, 1 Flowerpot, 1 Glasses, 1 Houseplant, 1 Laptop, 1 Mug, 1 Office building, 1 Pen, 1 Table, 2055.0ms
Speed: 12.0ms preprocess, 2055.0ms inference, 15.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250118212217
1 label saved to C:\Users\26073\Workspace\detector\_demo20250118212217\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250118212217.mp3


Traceback (most recent call last):
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\gradio\blocks.py", line 1589, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\anyio\_backends\_asyncio.py", line 2461, in run_sync_in_worker_thread
    re


image 1/1 C:\Users\26073\Workspace\detector\demo20250118212259.jpg: 480x640 (no detections), 1342.1ms
Speed: 4.0ms preprocess, 1342.1ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250118212259
0 label saved to C:\Users\26073\Workspace\detector\_demo20250118212259\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250118212259.mp3


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\middleware\errors.py", line 165, i

...


image 1/1 C:\Users\26073\Workspace\detector\demo20250118212927.jpg: 640x640 1 Coffee cup, 1 Desk, 1 Flowerpot, 1 Glasses, 1 Houseplant, 1 Laptop, 1 Mug, 1 Office building, 1 Pen, 1 Table, 1830.9ms
Speed: 10.0ms preprocess, 1830.9ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250118212927
1 label saved to C:\Users\26073\Workspace\detector\_demo20250118212927\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250118212927.mp3

image 1/1 C:\Users\26073\Workspace\detector\demo20250118213159.jpg: 480x640 1 Boy, 1 Clothing, 1 Human face, 1150.6ms
Speed: 5.0ms preprocess, 1150.6ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250118213159
1 label saved to C:\Users\26073\Workspace\detector\_demo20250118213159\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250118213159.mp3


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\middleware\errors.py", line 165, i


image 1/1 C:\Users\26073\Workspace\detector\demo20250118213214.jpg: 448x640 1 Glasses, 3 Human faces, 4 Mans, 1145.0ms
Speed: 7.0ms preprocess, 1145.0ms inference, 14.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250118213214
1 label saved to C:\Users\26073\Workspace\detector\_demo20250118213214\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250118213214.mp3

image 1/1 C:\Users\26073\Workspace\detector\demo20250118213417.jpg: 384x640 5 Clothings, 1 Girl, 5 Human faces, 3 Persons, 1003.0ms
Speed: 3.0ms preprocess, 1003.0ms inference, 13.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250118213417
1 label saved to C:\Users\26073\Workspace\detector\_demo20250118213417\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250118213417.mp3

image 1/1 C:\Users\26073\Workspace\detector\demo20250118213550.jpg: 384x640 5 Clothings, 1 Girl, 5 Human fac

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  File "C:\Users\26073\miniconda3\envs\gradio\lib\site-packages\starlette\middleware\errors.py", line 165, i


image 1/1 C:\Users\26073\Workspace\detector\demo20250118213601.jpg: 480x640 11 Foods, 1618.0ms
Speed: 3.1ms preprocess, 1618.0ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250118213601
1 label saved to C:\Users\26073\Workspace\detector\_demo20250118213601\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250118213601.mp3

image 1/1 C:\Users\26073\Workspace\detector\demo20250118214134.jpg: 480x640 13 Clothings, 12 Human faces, 5 Mans, 1 Table, 1676.1ms
Speed: 3.8ms preprocess, 1676.1ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to C:\Users\26073\Workspace\detector\_demo20250118214134
1 label saved to C:\Users\26073\Workspace\detector\_demo20250118214134\labels
语音合成成功，音频保存为 C:\Users\26073\Workspace\detector\demo20250118214134.mp3

image 1/1 C:\Users\26073\Workspace\detector\demo20250118214353.jpg: 480x640 1 Glasses, 1 Human face, 1 Person, 1532.7ms
Speed: 3.0ms p